In [2]:
%cd caddy_tailscale_sidecar

/home/griffin/git/griffinht.com/notebooks/caddy_tailscale_sidecar


# Deploying a VPS on Kubernetes

It doesn't matter whether the server here is an EC2 instance or 

The point is that we have a Linux machine with a public IP address that we can connect to.

todo guide lab on removing that public IP and replacing with tailscale

Here is a scenario that I think most software developers understand. Let's say you have a VPS. You 


The steps here are simple. Your VPS comes with a public IP address. You can access it 

I think that the file server is the most fundamental building block of a computer network. A safe place for you files. Entire businesses can be built off of nothing but a file server.

In [ ]:
womp wom

In [ ]:
Podman generate kuberentes pod from pod

Let's make a pod, and call it caddy.

In [53]:
%%writefile pod.yaml
apiVersion: v1
kind: Pod
metadata:
    name: caddy
spec:
    containers:
      - name: caddy
        image: caddy/caddy

Overwriting pod.yaml


In [56]:
%%sh
kubectl apply -f pod.yaml

pod/caddy created


In [48]:
%%sh
kubectl get pods 

NAME    READY   STATUS    RESTARTS   AGE
caddy   1/1     Running   0          27m


In [11]:
%%sh
kubectl exec caddy -- wget localhost:80 -O -

Connecting to localhost:80 (127.0.0.1:80)
writing to stdout
-                    100% |********************************| 12256  0:00:00 ETA
written to stdout


<!DOCTYPE html>
<html>
	<head>
		<title>Caddy works!</title>
		<meta charset="utf-8">
		<meta name="viewport" content="width=device-width, initial-scale=1.0">
		<link rel="icon" href="data:,">
		<style>
			* {
				box-sizing: border-box;
				padding: 0;
				margin: 0;
			}

			body {
				background: #f1f4f5;
				font-family: sans-serif;
				font-size: 20px;
				-webkit-font-smoothing: antialiased;
			}

			a {
				color: #2f79ff;
				text-decoration: none;
			}

			a:hover {
				text-decoration: underline;
			}

			.stack {
				width: 70%;
				max-width: 1150px;
				margin: 25px 0 150px 25px;
				display: flex;
				align-content: flex-start;
			}

			.paper {
				position: relative;
				flex-shrink: 0;
				width: 100%;

				background: white;
				border-radius: 3px;
				box-shadow: 1px 2px 4px 1px rgba(0, 0, 0, .15);
				padding: 100px;
			}

			#paper1 {
				top: 45px;
				left: 50px;
			}

			#paper2 {
				top: 20px;
				left: -100%;
			}

			#paper3 {
				top: 25px;
				transform: r

I can also "port forward" Caddy to my dev machine with `kubectl port-forward`.

In [57]:
%%sh
kubectl port-forward pod/caddy 8080:80

Forwarding from 127.0.0.1:8080 -> 80
Forwarding from [::1]:8080 -> 80
Handling connection for 8080
Process is interrupted.


With `kubectl port-forward` running, I can visit [http://localhost:8080](http://localhost:8080) from my laptop.

Let's use Tailscale instead:

In [157]:
%%writefile pod.yaml
apiVersion: v1
kind: Pod
metadata:
    name: caddy
spec:
    containers:
      - name: caddy
        image: caddy/caddy
      - name: tailscale
        image: ghcr.io/tailscale/tailscale:latest
        env:
          # disable Kubernetes secrets and use OAuth login instead
          - name: TS_KUBE_SECRET
            value: ""

Overwriting pod-tailscale.yaml


In [145]:
%%sh
kubectl apply -f pod.yaml

The Pod "caddy" is invalid: spec: Forbidden: pod updates may not change fields other than `spec.containers[*].image`,`spec.initContainers[*].image`,`spec.activeDeadlineSeconds`,`spec.tolerations` (only additions to existing tolerations),`spec.terminationGracePeriodSeconds` (allow it to be set to 1 if it was previously negative)
  core.PodSpec{
  	... // 7 identical fields
  	DNSPolicy:                    "ClusterFirst",
  	NodeSelector:                 nil,
- 	ServiceAccountName:           "default",
+ 	ServiceAccountName:           "tailscale",
  	AutomountServiceAccountToken: nil,
  	NodeName:                     "gk3-my-cluster-nap-1u2x0pda-22c5dacc-q2wk",
  	... // 21 identical fields
  }



CalledProcessError: Command 'b'kubectl apply -f pod-tailscale.yaml\n'' returned non-zero exit status 1.

I guess we can't add a container to an already running pod. No worries - we can delete the pod and recreate it:

In [158]:
%%sh
kubectl delete pod caddy

pod "caddy" deleted


In [159]:
%%sh
kubectl apply -f pod.yaml

pod/caddy created


Note that `kubectl apply --force` works great here as well.

In [161]:
%%sh
kubectl logs caddy tailscale

boot: 2024/06/23 21:24:06 Starting tailscaled
boot: 2024/06/23 21:24:06 Waiting for tailscaled socket
2024/06/23 21:24:06 logtail started
2024/06/23 21:24:06 Program starting: v1.68.1-t92eacec73, Go 1.22.4: []string{"tailscaled", "--socket=/tmp/tailscaled.sock", "--state=mem:", "--statedir=/tmp", "--tun=userspace-networking"}
2024/06/23 21:24:06 LogID: a9955dc974bf7e0f06c1a40c7f6083d53aec0458feddda2bd829c1dce08455de
2024/06/23 21:24:06 logpolicy: using system state directory "/var/lib/tailscale"
logpolicy.ConfigFromFile /var/lib/tailscale/tailscaled.log.conf: open /var/lib/tailscale/tailscaled.log.conf: no such file or directory
logpolicy.Config.Validate for /var/lib/tailscale/tailscaled.log.conf: config is nil
2024/06/23 21:24:06 dns: [rc=unknown ret=direct]
2024/06/23 21:24:06 dns: using "direct" mode
2024/06/23 21:24:06 dns: using *dns.directManager
2024/06/23 21:24:06 wgengine.NewUserspaceEngine(tun "userspace-networking") ...
2024/06/23 21:24:06 dns: using dns.noopManager
2024/06/

I just clicked the magic link and logged in on my dev machine. I could also have generated an auth key and passed it to the Tailscale pod, but I figured this would be easier.

In [165]:
%%sh
tailscale status | grep caddy

100.100.140.99  caddy                griffinht@   linux   -


Caddy is connected to my Tailnet!

Let's ping it from my laptop, which is also connected to my Tailnet.

In [167]:
%%sh
curl caddy

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed


<!DOCTYPE html>
<html>
	<head>
		<title>Caddy works!</title>
		<meta charset="utf-8">
		<meta name="viewport" content="width=device-width, initial-scale=1.0">
		<link rel="icon" href="data:,">
		<style>
			* {
				box-sizing: border-box;
				padding: 0;
				margin: 0;
			}

			body {
				background: #f1f4f5;
				font-family: sans-serif;
				font-size: 20px;
				-webkit-font-smoothing: antialiased;
			}

			a {
				color: #2f79ff;
				text-decoration: none;
			}

			a:hover {
				text-decoration: underline;
			}

			.stack {
				width: 70%;
				max-width: 1150px;
				margin: 25px 0 150px 25px;
				display: flex;
				align-content: flex-start;
			}

			.paper {
				position: relative;
				flex-shrink: 0;
				width: 100%;

				background: white;
				border-radius: 3px;
				box-shadow: 1px 2px 4px 1px rgba(0, 0, 0, .15);
				padding: 100px;
			}

			#paper1 {
				top: 45px;
				left: 50px;
			}

			#paper2 {
				top: 20px;
				left: -100%;
			}

			#paper3 {
				top: 25px;
				transform: r

100 12256  100 12256    0     0  78486      0 --:--:-- --:--:-- --:--:-- 79070


Everything works. The only caveat here is that this connection is ephemeral. When the pod restarts, the connection will be lost. Tailscale doesn't have a way to save the authentication token anywhere, which means I would have to click the magic link every time the pod restarts.

Let's fix that.

In [16]:
%%writefile pod.yaml
apiVersion: v1
kind: PersistentVolumeClaim
metadata:
  name: tailscale-state
spec:
  accessModes:
    # read/write from a single node
    # https://kubernetes.io/docs/concepts/storage/persistent-volumes/#access-modes
    - ReadWriteOnce
  # todo idk where the docs are for this - i just guessed
  resources:
    requests:
      storage: 10Mi
---
apiVersion: v1
kind: Pod
metadata:
    name: caddy
spec:
    containers:
      - name: caddy
        image: caddy/caddy
      - name: tailscale
        image: ghcr.io/tailscale/tailscale:latest
        env:
          # disable Kubernetes secrets and use OAuth login instead
          - name: TS_KUBE_SECRET
            value: ""
          - name: TS_STATE_DIR
            value: "/var/lib/tailscale"
        volumeMounts:
          - name: tailscale-state
            mountPath: "/var/lib/tailscale"
    volumes:
      - name: tailscale-state
        persistentVolumeClaim:
          claimName: tailscale-state

Overwriting pod.yaml


In [17]:
%%sh
kubectl apply -f pod.yaml

persistentvolumeclaim/tailscale-state created


pod/caddy created


In [21]:
%%sh
kubectl logs caddy tailscale

boot: 2024/06/23 22:20:49 Starting tailscaled
boot: 2024/06/23 22:20:49 Waiting for tailscaled socket
2024/06/23 22:20:49 logtail started
2024/06/23 22:20:49 Program starting: v1.68.1-t92eacec73, Go 1.22.4: []string{"tailscaled", "--socket=/tmp/tailscaled.sock", "--statedir=/var/lib/tailscale", "--tun=userspace-networking"}
2024/06/23 22:20:49 LogID: 1afd630dad531ee594c20bebc7a4677c805d4343fb4165ff7d3953f6528f43ba
2024/06/23 22:20:49 logpolicy: using system state directory "/var/lib/tailscale"
2024/06/23 22:20:49 dns: [rc=unknown ret=direct]
2024/06/23 22:20:49 dns: using "direct" mode
2024/06/23 22:20:49 dns: using *dns.directManager
2024/06/23 22:20:49 wgengine.NewUserspaceEngine(tun "userspace-networking") ...
2024/06/23 22:20:49 dns: using dns.noopManager
2024/06/23 22:20:49 link state: interfaces.State{defaultRoute=eth0 ifs={eth0:[10.15.0.88/26]} v4=true v6=false}
2024/06/23 22:20:49 onPortUpdate(port=45996, network=udp6)
2024/06/23 22:20:49 magicsock: [warning] failed to force-se

In [185]:
%%sh
curl caddy

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed


<!DOCTYPE html>
<html>
	<head>
		<title>Caddy works!</title>
		<meta charset="utf-8">
		<meta name="viewport" content="width=device-width, initial-scale=1.0">
		<link rel="icon" href="data:,">
		<style>
			* {
				box-sizing: border-box;
				padding: 0;
				margin: 0;
			}

			body {
				background: #f1f4f5;
				font-family: sans-serif;
				font-size: 20px;
				-webkit-font-smoothing: antialiased;
			}

			a {
				color: #2f79ff;
				text-decoration: none;
			}

			a:hover {
				text-decoration: underline;
			}

			.stack {
				width: 70%;
				max-width: 1150px;
				margin: 25px 0 150px 25px;
				display: flex;
				align-content: flex-start;
			}

			.paper {
				position: relative;
				flex-shrink: 0;
				width: 100%;

				background: white;
				border-radius: 3px;
				box-shadow: 1px 2px 4px 1px rgba(0, 0, 0, .15);
				padding: 100px;
			}

			#paper1 {
				top: 45px;
				left: 50px;
			}

			#paper2 {
				top: 20px;
				left: -100%;
			}

			#paper3 {
				top: 25px;
				transform: r

100 12256  100 12256    0     0  57250      0 --:--:-- --:--:-- --:--:-- 57271


					<!-- English --> <span class="lang">Congratulations!</span>
					<!-- Japanese --> <span class="lang">おめでとう!</span>
					<!-- Spanish --> <span class="lang">Felicidades!</span>
					<!-- Chinese --> <span class="lang">恭喜!</span>
					<!-- Hindi --> <span class="lang">बधाई हो!</span>
					<!-- Russian --> <span class="lang">Поздравляю! </span>
					<span class="emoji">🎊</span>
				</h1>

				<p>
					Your web server is working. Now make it work for you. 💪
				</p>
				<p>
					Caddy is ready to serve your site over HTTPS:
				</p>
				<ol>
					<li>Point your domain's A/AAAA DNS records at this machine.</li>
					<li>Upload your site's files to <code>/var/www/html</code>.
					<li>
						Edit your Caddyfile at <code>/etc/caddy/Caddyfile</code>:
						<ol>
							<li>Replace <code>:80</code> with your domain name</li>
							<li>Change the site root to <code>/var/www/html</code></li>
						</ol>
					</li>
					<li>Reload the configuration: <code>systemctl reload caddy</code></li>
	

Caddy still works, and will no longer be disconnected when the pod restarts. Success!

Next steps:
Proxy some HTTP services behind Caddy

Use Tailscale to expose many other services from many pods